In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
#import necessary modules for package

import wikipediaapi
import pandas as pd
import networkx as nx
import copy
from datetime import date
from itertools import zip_longest
from datetime import datetime

#used for within Jupyter notebook and for interactivity of networkx graphs
import pyvis
from pyvis import network as net

g = net.Network(notebook = True)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [2]:
#taking the more straightforward route, user supplies program with 2 wikipedia links themselves

search_node_a = input("enter first wikipedia page name: ")
search_node_b = input("enter second wikipedia page name: ")

enter first wikipedia page name: Miramar High School
enter second wikipedia page name: Åland War


In [3]:
#get Wikipedia page for the search nodes a, b

wiki_wiki = wikipediaapi.Wikipedia('en')

page_a = wiki_wiki.page(search_node_a)
page_b = wiki_wiki.page(search_node_b)

In [4]:
#f'ns used to get links from Wikipedia pages and clean page links list to remove Wikipedia attributes 

def return_links(page, list_n):
    list_n.clear()
    links = page.links
    for title in sorted(links.keys()):
        list_n.append(title)
            
def clean_links(list_node):
    for item in list_node.copy():
        if ((item in ('Wayback Machine', 'Specials (Unicode block)')) or (item.endswith(('(disambiguation)', '(identifier)'))) or (item.startswith(('List of ', 'Category:', 'File:', 'Help:', 'Talk:', 'Template:', 'Wikipedia:', 'Template talk:', 'Portal:', 'Wiki')))):
            list_node.remove(item)

In [5]:
list_node_a = []
list_node_b = []

return_links(page_a, list_node_a)
return_links(page_b, list_node_b)

clean_links(list_node_a)
clean_links(list_node_b)

In [6]:
list_node_a

['American Heritage School (Florida)',
 'American InterContinental University South Florida Campus',
 'Archbishop Edward A. McCarthy High School',
 'Asian people',
 'Athletic nickname',
 'Atlantic Technical Center',
 'Black people',
 'Blanche Ely High School',
 'Boyd H. Anderson High School',
 'Broward College',
 'Broward County',
 'Broward County Public Schools',
 'Broward County, Florida',
 'Broward Virtual Education High',
 'Cardinal Gibbons High School (Fort Lauderdale, Florida)',
 'Chaminade-Madonna College Preparatory School',
 'Charles W. Flanagan High School',
 'Cincinnati Enquirer',
 'City College (Florida)',
 'Clarence Coleman (gridiron football)',
 'Coconut Creek High School',
 'College Academy @ BC',
 'Cooper City High School',
 'Coral Glades High School',
 'Coral Springs Charter School',
 'Coral Springs High School',
 'Cypress Bay High School',
 'David Posnack Jewish Day School',
 'DeLand',
 'DeLand High School',
 'Dean Biasucci',
 'Deerfield Beach High School',
 'Defensiv

In [9]:
#add page title to graph dict as key and nodes as values
#form the network from the graph dict 

graph = {}

graph[(page_a.title)] = list_node_a
graph[(page_b.title)] = list_node_b

G = nx.to_networkx_graph(graph)

In [7]:
now = datetime.now()

page_links = []

error = ""
shortest_path_list = ""

combined_a_b = list_node_a + list_node_b

In [10]:
titles = combined_a_b

for t in titles:
    page = wiki_wiki.page(t)
    return_links(page, page_links)
    clean_links(page_links)
    
    graph[(page.title)] = copy.deepcopy(page_links)
    
    for p in page_links:
        if p not in titles:
            titles.append(p)
        else:
            break;
        break;
            
    page_links.clear();
    del page; 
    
    try:
        G = nx.to_networkx_graph(graph)
        nx.shortest_path(G, page_a.title, page_b.title)
    except nx.NetworkXNoPath:
        if t == titles[-1]: 
            error = "No path found between %s and %s" % (page_a.title, page_b.title)
            break;
        else:
            #print(t)
            continue;
    else:
        shortest_path_list = nx.shortest_path(G, page_a.title, page_b.title)
        break;

    if error != "":
        break;

if error == "":
    output = ("There are " + str(len(shortest_path_list) - 1) + " degrees of separation between " + page_a.title + " and " + page_b.title + "\n\n" + str(shortest_path_list))
else:
    output = error
    
print(output)
    
log = open("run_logs.txt", "a")
log.write(output + "\n")
log.write("Run: " + str(now) + "\n\n")
log.close()

There are 3 degrees of separation between Miramar High School and Åland War

['Miramar High School', 'Black people', 'American Revolutionary War', 'Åland War']


In [11]:
graph

{'Miramar High School': ['American Heritage School (Florida)',
  'American InterContinental University South Florida Campus',
  'Archbishop Edward A. McCarthy High School',
  'Asian people',
  'Athletic nickname',
  'Atlantic Technical Center',
  'Black people',
  'Blanche Ely High School',
  'Boyd H. Anderson High School',
  'Broward College',
  'Broward County',
  'Broward County Public Schools',
  'Broward County, Florida',
  'Broward Virtual Education High',
  'Cardinal Gibbons High School (Fort Lauderdale, Florida)',
  'Chaminade-Madonna College Preparatory School',
  'Charles W. Flanagan High School',
  'Cincinnati Enquirer',
  'City College (Florida)',
  'Clarence Coleman (gridiron football)',
  'Coconut Creek High School',
  'College Academy @ BC',
  'Cooper City High School',
  'Coral Glades High School',
  'Coral Springs Charter School',
  'Coral Springs High School',
  'Cypress Bay High School',
  'David Posnack Jewish Day School',
  'DeLand',
  'DeLand High School',
  'Dean

In [55]:
#titles - all nodes
#graph - all edges (have to transform)

#nx.draw(G)

#net = Network()

#g.from_nx(G)
#g.show("example.html")